In [1]:
from Bio import SeqIO
from collections import Counter
import pandas as pd 
import re
import requests
import json
import time
import itertools
from itertools import islice
import glob
import pprint
import seaborn as sns
import pylab as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import pycountry 
import pycountry_convert as pc
pp = pprint.PrettyPrinter()
import numpy as np
from tabulate import tabulate
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import regex as re
import plotly.graph_objects as go
import os


In [2]:
%set_env folder = X:\5_Research\Paul\dsi_origins\


env: folder=X:\5_Research\Paul\dsi_origins\


In [3]:
path = %env folder


In [7]:
df_scholarly = pd.read_csv(path+R'processed_data\scholarly.csv', index_col = 0, keep_default_na=False, na_values=[np.nan])

C:\Users\a494-nipw010\AppData\Local\Temp\ipykernel_4700\1148779120.py:1: DtypeWarning: Columns (2,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_scholarly = pd.read_csv(path+R'processed_data\scholarly.csv', index_col = 0, keep_default_na=False, na_values=[np.nan])


In [9]:
len(set(df_scholarly['id_select']))

96842

In [11]:
len(df_scholarly[df_scholarly['affiliation_country']!= ''])

292272

In [13]:
search_affiliations = []
for affiliation in list(df_scholarly['affiliation'][df_scholarly['affiliation_country'] == '']):
    if "Krefeld" in affiliation: 
        print(affiliation)
        search_affiliations.append(affiliation)

eChemisches und Veterinäruntersuchungsamt Rhein-Ruhr-Wupper, CVUA-RRW, Krefeld, Germany
Zoo Krefeld, Krefeld, Germany
Chemisches- und Veterinäruntersuchungsamt Rhein-Ruhr-Wupper, Krefeld, Germany
Staatliches Veterinäruntersuchungsamt Krefeld
4Department of Laboratory Medicine, HELIOS Hospital, Krefeld, Germany


In [15]:
#number of author listings where no country is known but an affilaition is known 
len(list(set(df_scholarly['affiliation'][(df_scholarly['affiliation_country'] == '') & (df_scholarly['affiliation'] != '')])))

71397

In [17]:
#
affiliations_to_check = list(set(df_scholarly['affiliation'][(df_scholarly['affiliation_country'] == '')& (df_scholarly['affiliation'] != '')]))
affiliations_foundcountries_dict = {}
affiliations = []
found_acountries = []
still_missing = []
several_countries_in_name_aff = []
several_countries_in_name_countries = []
names_in_names = ['Niger', 'Dominica', 'Guinea']

count = 1
for affiliation in affiliations_to_check: # in case of filtering for first authors & (df_scholarly['first_author'] == 'first_author')]) 
    ph = ''
    affiliations.append(affiliation)
    #print('\n\n'+str(count)) 
    #print(affiliation)
    
    
    
    if not pd.isna(affiliation): #some are just entirely unknown
        #print('\n\n'+str(count)) 
        #print(affiliation)
        countries_found_count = 0
        
        #first check if missing country name can be found in affiliation: 
        for country in list(pycountry.countries):
            if country.name in affiliation and 'Indiana University' not in affiliation and 'Indiana, United States' not in affiliation and 'Georgia State University' not in affiliation and 'Franceville' not in affiliation and 'New Mexico' not in affiliation and 'New Jersey' not in affiliation:
                countries_found_count = countries_found_count+1
                
                #in some cases, the second found country needs to be taken (because another country name is in the actual name)
                if country.name in names_in_names and 'Nigeria' in affiliation or 'Dominican Republic' in affiliation or 'Papua New Guinea' in affiliation:
                    #print(affiliation)
                    if 'Nigeria' in affiliation: 
                        ph = 'NG'
                        #print('Country_in_name (first of several):' + ph)
                    elif 'Dominican Republic' in affiliation: 
                        ph = 'DO'
                        #print('Country_in_name (first of several):' + ph)
                    elif 'Papua New Guinea' in affiliation: 
                        ph = 'PG'
                        #print('Country_in_name (first of several):' + ph)
                    break 
                
                else:
                    if countries_found_count >= 2: #if one country has already been found, break loop to take first mentioned country affiliation 
                        #several_countries_in_name_aff.append(affiliation) # this is just for getting country names within country names --> deactive  for actual labelling 
                        #several_countries_in_name_countries.append(country.name) # this is just for getting country names within country names --> deactive  for actual labelling 
                        #print(affiliation)
                        if 'Bonobo and Congo Biodiversity Initiative, Zoological Society of Milwaukee, Milwaukee' in affiliation: 
                            ph = 'US'
                            #print('Country_in_name (spelling):' + ph) 
                            break
                        elif "Thailand-Japan Research Collaboration Center on Emerging and Re-emerging Infections, Nonthaburi, Thailand" in affiliation or "Thailand-Japan Research Collaboration Center on Emerging and Re-emerging Infections (RCC-ERI), Nonthaburi 11000" in affiliation: 
                            ph = 'TH'
                            #print('Country_in_name (spelling):' + ph)
                            break
                        elif "Division of Virology, National Institute for Medical Research, Mill Hill, London, NW7 1AA, United Kingdom; Government Virus Unit, Queen Mary Hospital, Hong Kong SAR, People's Republic of China" in affiliation:
                            ph = 'GB'
                            #print('Country_in_name (first_of_several):' + ph) 
                            break
                        elif "Equatorial Guinea" in affiliation:
                            ph = 'GQ'
                            #print('Country_in_name (spelling):' + ph)
                            break
                        elif "Edinburgh, 51 Little France Crescent, Edinburgh EH16 4SA, United Kingdom" in affiliation: 
                            ph = 'GB'
                            #print('Country_in_name (first_of_several):' + ph) 
                            break
                        elif "Department of Paraclinical Sciences, Faculty of Veterinary Medicine, Norwegian University of Life Sciences, Ås, Norway; College of Veterinary Medicine" in affiliation:
                            ph = 'NO'
                            #print('Country_in_name (first_of_several):' + ph) 
                            break
                        elif "*University of Oxford, Molecular Immunology Group, Nuffield Department of Medicine and ‖Department of Cellular Science, John Radcliffe Hospital, Headington, Oxford OX3 9DU, United Kingdom; §Department of Anatomy, University of Kuopio" in affiliation: 
                            ph = 'GB'
                            #print('Country_in_name (first_of_several):' + ph) 
                            break
                        elif "Bamako, Mali; Centre National de Recherche et Formation sur le Paludisme, Ouagadougou, Burkina Faso" in affiliation: 
                            ph = 'ML'
                            #print('Country_in_name (first_of_several):' + ph) 
                            break
                        elif "Department of Plant Pathology, 4024 Throckmorton Plant Sciences Center" in affiliation: 
                            ph = 'US'
                            #print('Country_in_name (first_of_several):' + ph) 
                            break
                        elif "Department of Veterinary and Tropical Diseases, University of Pretoria, Pretoria, South Africa; Centro de Biotecnologia, Universidade Eduardo Mondlane, Mozambique" in affiliation: 
                            ph = 'ZA'
                            #print('Country_in_name (first_of_several):' + ph) 
                            break
                        elif "UMR Astre (Animals, Health, Territories, Risks, Ecosystems), Cirad, Inrae, Univ. Montpellier, 34398 Montpellier, France - Insect Pest Control Laboratory, Joint FAO/IAEA Centre of Nuclear Techniques in Food and Agriculture" in affiliation:
                            ph = 'FR'
                            #print('Country_in_name (first_of_several):' + ph) 
                            break
                        elif "State Institute of Hygiene, Warsaw, PolandWHO Regional Office for Europe, Copenhagen, Denmark" in affiliation: 
                            ph = 'PL'
                            #print('Country_in_name (first_of_several):' + ph) 
                            break
                        elif "From the Molecular Immunology Group, Tenovus Laboratory, Cancer Sciences Division, Southampton University Hospitals, Southampton, United Kingdom; the First Department of Internal Medicine and Medical Oncology, Wilhelminenspital, Vienna" in affiliation:
                            ph = 'GB'
                            #print('Country_in_name (first_of_several):' + ph) 
                            break
                        elif "Yunnan, China; Research Unit Tropical Mycology and Plants-Soil Fungi Interactions, Faculty of Agronomy, University of Parakou, BP 123 Parakou, Benin" in affiliation: 
                            ph = 'CN'
                            #print('Country_in_name (first_of_several):' + ph) 
                            break
                        elif "French Guiana, France" in affiliation or "French Guiana, Cayenne, France" in affiliation:
                            ph = 'GF'
                            #print('Country_in_name (political):' + ph) 
                            break
                        elif "Hong Kong" in affiliation and "China" in affiliation:
                            ph = 'HK'
                            #print('Country_in_name (political):' + ph) 
                            break
                        elif "Mayotte" in affiliation and 'Réunion' not in affiliation:
                            ph = 'YT'
                            #print('Country_in_name (political):' + ph) 
                            break
                        elif "Réunion, France" in affiliation:
                            ph = 'RE'
                            #print('Country_in_name (political):' + ph) 
                            break
                        elif "Guadeloupe" in affiliation:
                            ph = 'GP'
                            #print('Country_in_name (political):' + ph) 
                            break
            
                        else: 
                            #print('Country_in_name (first_of_several):' + ph)
                            break
                    else: #if we are still with the first encountered country, label with that 
                        ph = pc.country_name_to_country_alpha2(country.name)
                        #print('Country_in_name:' + ph)

                    
            else: 
                try: 
                    if country.common_name in affiliation: #only a few countries have such common names, thus 'try' statement 
                        #print(affiliation)
                        countries_found_count = countries_found_count+1
                        if countries_found_count >= 2: 
                            break
                        else: 
                            ph = pc.country_name_to_country_alpha2(country.name)
                            #print('Country_in_name_common:' + ph)    
                        
                except: 
                    pass
                
        if str(ph) == '': 
            if ', UK ' in affiliation or ', UK,' in affiliation or 'National Institute for Medical Research' in affiliation or 'London, England' in affiliation or 'Public Health England' in affiliation or 'Veterinary Laboratories Agency' in affiliation or 'Stevenage, England' in affiliation or 'UCL Institute of Ophthalmology' in affiliation or 'Strathclyde Institute of Pharmacy and Biomedical Sciences' in affiliation or 'Smith, Kline & French' in affiliation or 'Edward Jenner Institute for Vaccine Research' in affiliation or 'Department of Zoology;' in affiliation or 'Addenbrooke' in affiliation or ',UK' in affiliation or 'U.K.' in affiliation or 'Aberdeen' in affiliation or 'The Roslin Institute' in affiliation or 'Wellcome/CRC Institute for Developmental Biology and Cancer Research' in affiliation or 'Harwell Science and Innovation Campus' in affiliation or ', UK.' in affiliation or ', UK' in affiliation: 
                ph = 'GB'
                #print('Country_via_rule:' + ph)
            elif ', USA' in affiliation or 'Western University College of Veterinary Medicine' in affiliation or 'International Institute of Minnesota' in affiliation or 'Ohio Agricultural Research and Development Center' in affiliation or 'Walter Reed Army Institute of Research' in affiliation or 'Broad Institute of Massachusetts' in affiliation or 'Hobart Corporation' in affiliation or 'Departments of Civil and Environmental Engineering' in affiliation or 'Departments of aCivil and Environmental Engineering and' in affiliation or 'Procter & Gamble' in affiliation or 'Human Genome Sequencing Center' in affiliation or 'University of Oregon' in affiliation or 'University of Texas' in affiliation or 'Verenium Corporation' in affiliation or "Department of Medicine, Brigham and Women's Hospital" in affiliation or 'Edgewood Chemical Biological Center' in affiliation or 'Genome Science Group, Bioscience Division, Los Alamos National Laboratory, Los Alamos, NM 87545; and' in affiliation or 'Forsyth' in affiliation or 'Animal and Comparative Sciences' in affiliation or 'Department of Molecular and Cell Biology and the Center for Integrative Genomics' in affiliation or "All Children's Hospital" in affiliation or 'University of Rhode Island' in affiliation or 'Pacific Biosciences' in affiliation or 'International Sleep Products Association' in affiliation or 'Defenders of Wildlife' in affiliation or 'Texas Tech University' in affiliation or 'Pasadena, CA' in affiliation or 'Cleveland Clinic Lerner Research Institute' in affiliation or 'RI02809 USA' in affiliation or 'The Institute for Genomic Research, Rockville' in affiliation or 'Kosan Biosciences' in affiliation or 'National Jewish Center for Immunology and Respiratory Medicine' in affiliation or 'Sanford-Burnham Institute for Medical Research' in affiliation or 'La Jolla, CA' in  affiliation or 'University of Maine' in affiliation or 'Delaware Biotechnology Institute' in affiliation or 'Hawai' in affiliation or 'City University of New York' in affiliation or 'Invitrogen' in affiliation or 'University Hospital, Boston' in affiliation or 'National Museum of Natural History, Smithsonian Institution' in affiliation or 'Florida State University' in affiliation or 'New York State Psychiatric Institute' in affiliation or 'Pennsylvania' in affiliation or 'Roche Institute of Molecular Biology' in affiliation or 'Virginia Bioinformatics Institute' in affiliation or 'Departments of Microbiology and Thermal Biology Institute and' in affiliation or 'Nereus Pharmaceuticals' in affiliation or 'TigerLogic' in affiliation or 'From the Columbia River Inter-Tribal Fish Commission' in affiliation or 'College of Oceanic and Atmospheric Sciences, and' in affiliation or 'Florida A&M University' in affiliation or 'University of Dayton' in affiliation or 'Saint Petersburg, FL' in affiliation or 'St Petersburg, FL' in affiliation or 'Departments of Civil and Environmental Engineering,' in affiliation or 'Limnology and Marine Sciences Program' in affiliation or 'Salt Lake City' in affiliation or 'La Jolla, California' in affiliation or 'The Field Museum, Chicago' in affiliation or 'Icos' in affiliation or 'Ocean Genome Legacy, Inc' in affiliation or 'National Center for Agricultural Utilization Research' in affiliation or 'Division of Infectious Diseases and Department of Internal Medicine' in affiliation or 'Hospital Oakland Research Institute' in affiliation or 'La jolla, California' in affiliation or 'Yale University' in affiliation or 'University of Wisconsin' in affiliation or 'Maryland' in affiliation or 'Harvard University' in affiliation or 'University of Illinois' in affiliation or 'University of Washington' in affiliation or 'Stanford University' in affiliation or 'University of Chicago' in affiliation or 'Kewalo Marine Laboratory' in affiliation or 'University of California' in affiliation or 'Virginia Institute of Marine Science' in affiliation or 'University of Southern California' in affiliation or 'University of Pittsburgh' in affiliation or 'Columbia University' in affiliation or 'California Institute of Technology' in affiliation or 'University of Colorado' in affiliation or 'Craig Venter Institute' in affiliation or 'Josephine Bay Paul Center' in affiliation or 'Oregon State University' in affiliation or 'Maryland Department of Natural Resources Police' in affiliation or 'New Jersey' in affiliation or 'Palo Alto, California' in affiliation or 'University of Massachusetts' in affiliation or 'University of Miami School of Medicine' in affiliation or 'USA.' in affiliation or 'University of South Florida' in affiliation or 'Fox Chase Cancer Center' in affiliation or 'University of Pennsylvania' in affiliation or 'Chesapeake Biological Laboratory' in affiliation or 'Southampton' in affiliation or 'Washington University' in affiliation or 'Helen Wills Neuroscience Institute' in affiliation or 'California State University' in affiliation or 'Rosenstiel School of Marine and Atmospheric Science' in affiliation or 'US Department of Energy Joint Genome Institute' in affiliation or 'Monterey Bay Aquarium Research Institute' in affiliation or 'U. S. A.' in affiliation or 'U.S.A.' in affiliation or 'Kalamazoo College' in affiliation or 'Whitney Laboratory for Marine Bioscience' in affiliation or 'Baylor College of Medicine' in affiliation or 'Benaroya Research Institute' in affiliation or 'San Diego' in affiliation or 'Washington State University Vancouver' in affiliation or 'Oklahoma State University–Stillwater' in affiliation or 'University of Hawaii' in affiliation or 'Drexel University' in affiliation or 'University of Maryland' in affiliation or 'Worcester Foundation for Biomedical Research' in affiliation or 'Indiana University' in affiliation or 'National Cancer Institute, Frederick' in affiliation: 
                ph = 'US'
                #print('Country_via_rule:' + ph)
            elif ', Russia' in affiliation or 'Russian' in affiliation or 'Pacific Institute' in affiliation or 'Petersburg Nuclear Physics Institute' in affiliation or 'SibEnzyme' in affiliation or 'Российской академии наук' in affiliation or 'Virginia Institute of Marine Science' in affiliation or 'Moscow' in affiliation or 'All-Union Research Scientific Institute' in affiliation: 
                ph = 'RU'
                #print('Country_via_rule:' + ph)
            elif 'Republic of Korea' in affiliation or 'East Sea Fisheries Research Institute' in affiliation or 'Ulsan University' in affiliation or ', Korea' in affiliation or 'Yosu National University' in affiliation or 'Korea Polar Research Institute' in affiliation or 'Chungbuk University' in affiliation or 'Korea.' in affiliation or 'National Fisheries Research & Development Institute' in affiliation:
                ph = 'KR'
                #print('Country_via_rule:' + ph)
            elif ' Finnish Food Safety Authority' in affiliation: 
                ph = 'FI'
                #print('Country_via_rule:' + ph)
            elif 'Bundeswehr Institute of Microbiology' in affiliation or 'Uiversitat Munchen' in affiliation or 'Developmental Biology Unit,,' in affiliation or 'Merck & Co' in affiliation or 'Abteilung Membranbiochemie' in affiliation or 'Albrecht-von-Haller-Institut' in affiliation or 'Kieler Wirkstoff-Zentrum am IFM-GEOMAR' in affiliation or 'Techn. Universitat Munchen' in affiliation or 'GSF—National Research Center for Environment and Health' in affiliation or 'Christians Albrechts Universität' in affiliation or 'Technische Universität, München' in affiliation or 'Institute of Integrative Biology (IBZ)' in affiliation or 'Lüneburg University' in affiliation or 'Heidelberg' in affiliation or 'Forschungsinstitut für die Biologie landwirtschaftlicher Nutztiere' in affiliation or 'Botanisches Institut' in affiliation or 'University Hamburg' in affiliation or 'Helmholtz Center for Infection Research' in affiliation: 
                ph = 'DE'
                #print('Country_via_rule:' + ph)
            elif 'Buenos Aires' in affiliation: 
                ph = 'AR'
                #print('Country_via_rule:' + ph)
            elif 'UPRRP College of Natural Sciences' in affiliation: 
                ph = 'PR'
                #print('Country_via_rule:' + ph)
            elif 'University of Tromsø' in affiliation or 'Unifob Petroleum' in affiliation or 'Bergen' in affiliation or 'Hedmark University College' in affiliation or 'University of Nordland' in affiliation or 'Norwegian' in affiliation:
                ph = 'NO'
                #print('Country_via_rule:' + ph)
            elif 'Pierre-and-Marie-Curie University' in affiliation or 'University of Auvergne' in affiliation or 'Ecologie, Evolution, Symbiose' in affiliation or 'aCentre National de la Recherche Scientifique, Unité Mixte de Recherche 7232 and' in affiliation or "Institut méditerranéen de biodiversité et d'écologie marine et continentale" in affiliation or 'SupAgro' in affiliation or 'Modèles en biologie cellulaire et évolutive' in affiliation or 'Muséum National d’Histoire Naturelle' in affiliation or 'SAMU Social' in affiliation or 'Institute for the Protection and Security' in affiliation or 'France FR' in affiliation or 'Station de Biologie Marine du Muséum National d’Histoire Naturelle' in affiliation or 'Université Pierre' in affiliation or 'UMR 958' in affiliation or 'Marion Merrell Dow' in affiliation or 'Equipe Ecophysiologie: Adaptation et Evolution Moléculaires' in affiliation or 'Institut de Génétique et Microbiologie and' in affiliation or 'Écologie et Paléontologie' in affiliation or 'Développement et évolution ' in affiliation or 'Adria Airways' in affiliation or 'ANSES' in affiliation or 'Blaise Pascal University' in affiliation or 'Nancy-Université' in affiliation or 'university of lille' in affiliation or 'Laboratoire de Bactériologie' in affiliation or 'University of Provence' in affiliation or 'Université de Pau' in affiliation or 'University of the Mediterranean' in affiliation or 'Biologie des Organismes et Ecosystèmes Aquatiques' in affiliation or 'University of Rennes' in affiliation or 'Mer et santé' in affiliation or 'Villefranche-sur-mer' in affiliation:
                ph = 'FR'
                #print('Country_via_rule:' + ph)
            elif 'Monash University' in affiliation or 'Museum of Tropical Queensland' in affiliation or 'Cooperative Research Centre' in affiliation or 'Queensland Department of Primary Industries' in affiliation or 'Queensland Biosciences Precinct' in affiliation: 
                ph = 'AU'
                #print('Country_via_rule:' + ph)
            elif 'México' in affiliation or 'CINVESTAV' in affiliation or 'Escuela Nacional de Ciencias Biológicas (ENCB)' in affiliation or 'Universidad Autónoma de Yucatán' in affiliation or 'Centro de Investigación en Alimentación y Desarrollo' in affiliation or 'Centro de Investigacion en Alimentacion y Desarrollo' in affiliation: 
                ph = 'MX'
                #print('Country_via_rule:' + ph)
            elif 'Japan, JP' in affiliation or 'Tokyo' in affiliation or 'Nagasaki' in affiliation or '*Biochemistry and' in affiliation or 'Kazusa DNA Research Institute' in affiliation or 'Department of Bacterial Infections1 and' in affiliation or '*Subground Animalcule Retrieval Program and' in affiliation or 'Gulf Coast Regional Blood Center' in affiliation or 'Fisheries Research Agency' in affiliation or 'Marine Biotechnology Institute (MBI)' in affiliation or 'RIKEN' in affiliation or 'Yamada Junior College of Home Economics' in affiliation or 'Amase Institute, Hayashibara Biochemical' in affiliation or 'Subground Animalcule Retrieval (SUGAR) Project' in affiliation or 'Suntory Institute for Bioorganic Research' in affiliation or 'Hokkaido Universit' in affiliation or 'National Research Institute of Fisheries Science, Yokohama' in affiliation or 'Okazaki Institute for Integrative Bioscience' in affiliation or 'Nichirei Corporation' in affiliation or 'Institute for Fermentation' in affiliation or 'Hitec' in affiliation or 'Saga Group' in affiliation or 'Nihon Shokuhin Kako' in affiliation or 'Laboratory of QC and Food Analysis' in affiliation or 'Division of Medical Genetics and Biochemistry' in affiliation or 'Mitsubishi' in affiliation or 'National Presto Industries' in affiliation or 'Tsukuba' in affiliation or 'Osaka' in affiliation or 'UniversityKochi' in affiliation or 'Sophia UniversityTokyo' in affiliation or 'Nippon Suisan Kaisha' in affiliation or 'Suntory Limited Research Center' in affiliation or 'Nagasaki Prefectural Institute of Fisheries' in affiliation or 'Tohoku UniversitySendai' in affiliation or 'National Research Institute of Fisheries and Environment of Inland Sea' in affiliation or 'Tohoku National Fisheries Research Institute' in affiliation or 'Iwate' in affiliation or 'Shinagawa Aquarium' in affiliation or 'Miyazaki University' in affiliation or 'Subground Animalcule Retrieval (SUGAR) Program' in affiliation: 
                ph = 'JP'
                #print('Country_via_rule:' + ph)
            elif 'National Institute of Education' in affiliation: 
                ph = 'SG'
                #print('Country_via_rule:' + ph)
            elif 'University of the Algarve' in affiliation or 'Instituto Nacional de Saúde Dr. Ricardo Jorge' in affiliation or 'Português' in affiliation or 'Lisbon' in affiliation or 'ULisboa' in affiliation or ' Lisboa' in affiliation or 'Departamento de Oceanografia e Pescas' in affiliation or 'Universidade Nova de Lisboa' in affiliation or 'Instituto de Biologia Molecular e Celular' in affiliation: 
                ph = 'PT'
                #print('Country_via_rule:' + ph)
            elif 'Ontario Veterinary College' in affiliation or 'Halifax' in affiliation or 'NRC Institute for Marine Biosciences' in affiliation or 'BC Research Inc' in affiliation or 'Université du Québec' in affiliation or 'Université du Québec à Montréal' in affiliation or 'Marine Institute of Memorial University of Newfoundland' in affiliation: 
                ph = 'CA'
                #print('Country_via_rule:' + ph)
            elif 'Östersidan' in affiliation: 
                ph = 'SE'
                #print('Country_via_rule:' + ph)
            elif 'Andrés Bello National University' in affiliation or 'Life Sciences School' in affiliation or 'Fundación Ciencia para la Vida' in affiliation or 'Pontifical Catholic University of Valparaíso' in affiliation: 
                ph = 'CL'
                #print('Country_via_rule:' + ph)
            elif 'Basel Institute for Immunology' in affiliation or 'Genentech' in affiliation or 'Swiss Institute of Allergy and Asthma Research' in affiliation or 'ISREC' in affiliation or 'Zürich' in affiliation or 'Friedrich Miescher Institute for Biomedical Research' in affiliation: 
                ph = 'CH'
                #print('Country_via_rule:' + ph)
            elif 'Sathabama University' in affiliation or 'National Centre for Biological Sciences' in affiliation or 'National Research Centre on Plant Biotechnology, LBS Building, Pusa Campus' in affiliation or 'Thiagarajar College' in affiliation or 'Stella Maris College' in affiliation or 'Calcutta' in affiliation or 'Nalco Holding Company' in affiliation or 'Kolkata' in affiliation or 'National Centre for Antarctic Ocean Research' in affiliation: 
                ph = 'IN'
                #print('Country_via_rule:' + ph)
            elif 'Institute of Business & Medical Careers' in affiliation or 'ANFACO-CECOPESCA' in affiliation or 'Centro de Investigacións Mariñas' in affiliation or 'University of Castilla' in affiliation or 'Generalitat of Catalonia' in affiliation or 'Área de Microbiología' in affiliation or 'Instituto Potosino de Investigación Científica y Tecnológica' in affiliation or 'Universidad Miguel Hernández de Elche' in affiliation or 'ICM Partners' in affiliation or 'Miguel Hernández University' in affiliation or 'Instituto Español de Oceanografía' in affiliation or 'España' in affiliation or 'Instituto Nacional de Técnica Aeroespacial' in affiliation: 
                ph = 'ES'
                #print('Country_via_rule:' + ph)
            elif 'Catholic University of Leuven' in affiliation or 'University of Hasselt' in affiliation or 'Free University of Brussels' in affiliation or 'Université de Namur' in affiliation: 
                ph = 'BE'
                #print('Country_via_rule:' + ph)
            elif 'Czech Republic' in affiliation or 'Prague' in affiliation: 
                ph = 'CZ'
                #print('Country_via_rule:' + ph)
            elif 'Lazzaro Spallanzani' in affiliation or 'Fondazione Don Carlo Gnocchi Onlus' in affiliation or 'CONISMA ULR' in affiliation or 'Seconda Università degli Studi di Napoli' in affiliation or 'Università di Napoli' in affiliation or 'Pozzuoli (Napoli)' in affiliation or 'IRSA' in affiliation or 'Istituto di Biomedicina' in affiliation or 'Italia' in affiliation or 'CNR-ISE Istituto per lo Studio degli Ecosistemi' in affiliation or 'CNR-ICVBC Istituto per la Conservazione e la Valorizzazione dei Beni Culturali' in affiliation or 'Istituto di Biomedicina e Immunologia Molecolare “Alberto Monroy”' in affiliation or 'University of Genova' in affiliation or 'Istituto Nazionale per la Ricerca sul Cancro' in affiliation: 
                ph = 'IT'
                #print('Country_via_rule:' + ph)
            elif 'Evandro Chagas Institute' in affiliation or 'Universidade Federal de Lavras' in affiliation or 'Maringá' in affiliation or 'São Paulo' in affiliation or 'Escola Superior de Agricultura Luiz de Queiroz' in affiliation or 'Ribeirão Preto' in affiliation or 'Brasil' in affiliation or 'LABOMAR' in affiliation or 'Laboratório de Genética Microbiana' in affiliation or 'UNIVALI' in affiliation or 'Federal University of Ceará' in affiliation or 'Empresa Brasileira' in affiliation or 'Pontifícia Universidade Católica do Rio Grande do Sul' in affiliation: 
                ph = 'BR' 
                #print('Country_via_rule:' + ph)
            elif 'Seibersdorf' in affiliation or 'Universität für Bodenkultur' in affiliation: 
                ph = 'AT'
                #print('Country_via_rule:' + ph)
            elif 'Kathmandu' in affiliation: 
                ph = 'NP'
                #print('Country_via_rule:' + ph)
            elif 'Slovak' in affiliation: 
                ph = 'SK'
            elif 'University of Ouargla' in affiliation or 'Bejaia, Algérie' in affiliation: 
                ph = 'DZ'
                #print('Country_via_rule:' + ph)
            elif 'Xiamen University' in affiliation or 'Zhejiang University' in affiliation or '1Ministry of Education Key Laboratory for Biodiversity Science and Ecological Engineering, and Insti' in affiliation or 'CAS - Institute of Genetics and Developmental Biology' in affiliation or 'Crops Research Institute' in affiliation or 'Institute of Aquatic Economic Animals' in affiliation or 'College of Fisheries' in affiliation or 'Tianjin Medical College' in affiliation or 'Liaoning Ocean and Fisheries Science Research Institute' in affiliation or 'Southern University of Science and Technology' in affiliation or 'State Key Laboratory Breeding Base of Marine Genetic Resources' in affiliation or 'Yunnan Agriculture University' in affiliation or 'Nankai University' in affiliation: 
                ph = 'CN'
                #print('Country_via_rule:' + ph)
            elif 'Danish' in affiliation or 'danish' in affiliation or 'Palsbøll lab' in affiliation or 'Royal Veterinary and Agricultural University' in affiliation or 'Odense University' in affiliation: 
                ph = 'DK'
                #print('Country_via_rule:' + ph)
            elif 'Finnish' in affiliation: 
                ph = 'FI'
                #print('Country_via_rule:' + ph)
            elif 'Iran' in affiliation:
                ph = 'IR'
                #print('Country_via_rule:' + ph)
            elif 'Turkey' in affiliation: 
                ph = 'TR'
            elif 'University of Port Elizabeth' in affiliation: 
                ph = 'ZA' 
                #print('Country_via_rule:' + ph)
            elif 'University of Manouba' in affiliation or 'Sfax, Tunisie' in affiliation: 
                ph = 'TN'
                #print('Country_via_rule:' + ph)
            elif 'Panamá' in affiliation:
                ph = 'PA'
                #print('Country_via_rule:' + ph)
            elif 'Tunceli University' in affiliation: 
                ph = 'TK'
                #print('Country_via_rule:' + ph)
            elif 'Perú' in affiliation: 
                ph = 'PE'
                #print('Country_via_rule:' + ph)
            elif 'Departments of aEarth and Planetary Sciences,' in affiliation:
                ph = 'IL'
                #print('Country_via_rule:' + ph)
            elif 'Hung Kuang Institute of Technology' in affiliation or 'Biodiversity Research Center' in affiliation: 
                ph = 'TW'
                #print('Country_via_rule:' + ph)
            elif 'Industrial Research Limited' in affiliation or 'Auckland War Memorial Museum' in affiliation or 'Bay of Plenty Polytechnic' in affiliation:  
                ph = 'NZ'
                #print('Country_via_rule:' + ph)
            elif 'Van Elsas lab' in affiliation or 'Olsen lab' in affiliation or 'Naturalis' in affiliation or 'Centraalbureau voor Schimmelcultures' in affiliation or 'Leiden' in affiliation or 'Wageningen Agricultural University' in affiliation: 
                ph = 'NL'
                #print('Country_via_rule:' + ph)
            elif 'University of Science and Technology of Hanoi' in affiliation: 
                ph = 'VN'
                #print('Country_via_rule:' + ph)
            elif 'Taylors University' in affiliation: 
                ph = 'MY'
                #print('Country_via_rule:' + ph)
            elif 'Grupo de Genética, Centro Nacional de' in affiliation: 
                ph = 'CU'
                #print('Country_via_rule:' + ph)
            elif 'Reykjavik' in affiliation: 
                ph = 'IS'
                #print('Country_via_rule:' + ph)
            elif 'Nicolaus Copernicus University' in affiliation: 
                ph = 'PL'
                #print('Country_via_rule:' + ph)
            elif 'Foundation for Research & Technology – Hellas' in affiliation: 
                ph = 'GR'
                #print('Country_via_rule:' + ph)
            elif 'Ruđer Bošković Institute' in affiliation: 
                ph = 'HR'
                #print('Country_via_rule:' + ph)
            elif 'Hungarian' in affiliation: 
                ph = 'HU'
                #print('Country_via_rule:' + ph)
            elif 'Yaoundé' in affiliation or 'Yaounde' in affiliation or 'Cameroon' in affiliation: 
                ph = 'CM'
            elif 'Gabon' in affiliation or 'Franceville' in affiliation:
                ph = 'GA'
                #print('Country_via_rule:' + ph)
            elif 'International Trademark Association' in affiliation or 'Facultad de Ciencias Exactas y Naturales' in affiliation: 
                ph = ''   
            else: 
                
                #if the same affiliaiton has a country in another datapoint and if those countries are not conflicting, label affiliation with the known country info
                if len(df_scholarly[(df_scholarly['affiliation'] == affiliation) & (df_scholarly['affiliation_country'] != '')]) > 0 and len(set(df_scholarly['affiliation_country'][(df_scholarly['affiliation'] == affiliation) & (df_scholarly['affiliation_country'] == '')])) > 0:
                    ph = list(set(df_scholarly['affiliation_country'][(df_scholarly['affiliation'] == affiliation) & (df_scholarly['affiliation_country'] != '')]))[0]
                    #print('Country_via_other_datapoint:' + ph)
                
                
                # else leave it 
                else: 
                    still_missing.append(affiliation)
                    #print('missing')
    #labl as still missing if they do not even have an affiliation reported
    else: 
        still_missing.append(affiliation)
        #print('missing')
        
    
    found_acountries.append(ph)
    affiliations_foundcountries_dict[affiliation] = ph
    #print('added: ' + ph)
    
    count = count +1
    


In [18]:
# create new dataframe to include the newly found
df_edit = df_scholarly[df_scholarly['affiliation_country'] == '']
df_edit['affiliation_country'] = df_edit['affiliation'].map(affiliations_foundcountries_dict)
df_edit['edited_aff'] = 1

# merge both 
df_scholarly_edit = pd.merge(df_scholarly[df_scholarly['affiliation_country'] != ''],df_edit, how = 'outer') # add those that already had country to those that have a new country
df_scholarly_edit['edited_aff'] = df_scholarly_edit[['edited_aff']].fillna(0)

C:\Users\a494-nipw010\AppData\Local\Temp\ipykernel_4700\3773533740.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edit['affiliation_country'] = df_edit['affiliation'].map(affiliations_foundcountries_dict)
C:\Users\a494-nipw010\AppData\Local\Temp\ipykernel_4700\3773533740.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edit['edited_aff'] = 1


In [19]:
Counter(df_scholarly_edit['affiliation'][df_scholarly_edit['affiliation_country'] == '']).most_common()

[('International Trademark Association', 314),
 ('Facultad de Ciencias Exactas y Naturales', 136),
 ('Primate Research Institute', 66),
 ('School of Biological Sciences', 61),
 ('Scottish Crop Research Institute', 60),
 ('Public Health Research Institute', 58),
 ('National Chemical Laboratory', 56),
 ('Federal University of Uberlandia', 56),
 ('Center for Infectious Disease Research and Policy', 55),
 ('CEVA Logistics', 54),
 ('National Microbiology Laboratory', 54),
 ('Instituto Superior de Agronomia', 51),
 ('Microbiology And Immunology', 50),
 ('Food and Agriculture Organization', 50),
 ('National Agricultural Research Centre', 48),
 ('Illinois Natural History Survey', 48),
 ('SIDI', 48),
 ('College of Horticulture', 47),
 ('Federal University of Tocantins', 47),
 ('Macaulay Institute', 47),
 ('Department of Microbiology', 46),
 ('Monogram Biosciences', 46),
 ('National Resource Center', 45),
 ('Boyce Thompson Institute for Plant Research', 43),
 ('Central Science Laboratory', 42),


In [20]:
#add the affiliation group to the affiliation country
df_country2grp = pd.read_csv(path+R'raw_data\2024-08-25_country2grp.csv.gz',keep_default_na=False, na_values=[''])
df_country = pd.read_csv(path+R'raw_data\2024-08-25_country.csv.gz',keep_default_na=False, na_values=[''])

#rduce to main groups to simplify
main_groups = ['G77', 'OECD', 'BRICS']
df_country2grp_ed = df_country2grp.loc[df_country2grp['GRP'].isin(main_groups)]

#create dictionaries: (country name -->) iso --> group
country_iso = dict(zip(list(df_country['NAME']),list(df_country['ISO_STATE'])))
country_iso['Taiwan'] = 'TW'

iso_group_dict = dict(zip(list(df_country2grp_ed['ISO']),list(df_country2grp_ed['GRP'])))
iso_group_dict['IN'] = 'BRICS'
iso_group_dict['ZA'] = 'BRICS'
iso_group_dict['BR'] = 'BRICS'
iso_group_dict['CL'] = 'G77'
iso_group_dict['CO'] = 'G77'
iso_group_dict['CR'] = 'G77'
iso_group_dict['TW'] = np.NaN




df_scholarly_edit['GROUP_lit'] = df_scholarly_edit['affiliation_country'].map(iso_group_dict)


In [25]:
#number of publications with at least one known affiliation country
len(set(df_scholarly_edit['id_select'][df_scholarly_edit['affiliation_country'] != '']))

96230

In [27]:
len(set(df_scholarly['id_select'][df_scholarly['affiliation_country'] != '']))

84860

In [29]:
#number of publications with at least one known affiliation economic group 
len(set(df_scholarly_edit['id_select'][df_scholarly_edit['GROUP_lit'].notna()]))

94773

In [31]:
pubs_known_lastauth_country = set(df_scholarly_edit['id_select'][(df_scholarly_edit['GROUP_lit'].notna()) & (df_scholarly_edit['last_author'] == 1)])
len(pubs_known_lastauth_country)

89091

In [33]:
pubs_known_firstauth_country = set(df_scholarly_edit['id_select'][(df_scholarly_edit['GROUP_lit'].notna()) & (df_scholarly_edit['first_author'] == 1)])
len(pubs_known_firstauth_country)

92430

In [35]:
#nr of publication where at least one of the lead authors (first or last) economic group affiliations are known
pubs_known_first_or_last_auth_country = pubs_known_lastauth_country | pubs_known_firstauth_country
len(pubs_known_first_or_last_auth_country)

93617

In [47]:
df_scholarly_edit.to_csv(path+R'processed_data\scholarly_aff.csv')

In [48]:
df_scholarly_edit_groups = df_scholarly_edit[df_scholarly_edit['GROUP_lit'].notna()]

In [49]:
df_scholarly_edit_groups.to_csv(path+R'processed_data\scholarly_aff_group.csv')

In [50]:
# save just those that I edited for sharing
df_aff_edit = df_edit[['affiliation','affiliation_country']].drop_duplicates().reset_index(drop = True)
df_aff_edit['count'] = df_aff_edit['affiliation'].map(dict(Counter(df_edit['affiliation']).most_common()))
df_aff_edit.sort_values(by='count',ascending=False).to_csv(path+R'processed_data\scholarly_aff_editing.csv')